In [ ]:
# import dependencies 
import pandas as pd
import datetime as dt
import pymysql as sql

In [ ]:
def account_login(cur,conn):
    """
        Method Name: account_login
        Input:  cur=Cursor,
                conn= Server Connection
        Description: This function register the new user into the database and allows the regular user to login into the database.
        Output: name= User name,
                passwd= Account Password,
                customer_id= Account Number
    """
    print('=========================WELCOME TO Your Bank============================================================\n')
    print(dt.datetime.now())
    print('\n\033[1mPlease choose one of the following: \033[0m')
    # Initialize parameters to count the user attemts
    reg_pass_attempt = 0
    pass_attempt = 0
    user_attempt = 0 
    while True:
        print('1.New Customer Registration')
        print('2.Customer Login\n')
        # Get user input
        reg_ch1 = input('Please enter your choice here:')
        if reg_ch1=='1':# loop for customer registration
            while True:
                name = input('Enter your full name=\n') # Get the user name
                passwd = input('Enter your 4 digit password=\n') # Get password for the account
                if len(passwd)==4: # Confirm the password length
                    # Get the unique id number based on the database length, which later utilized to assign account number
                    id_nos = cur.execute('select * from user_table')
                    try:
                        # Update the fields of the main table based on the provided information
                        cur.execute('insert into user_table (ac_no,username,password) values ({c},"{a}","{b}")'.format(a=name, b=passwd, c=id_nos))
                        cur.execute('update user_table set login_fail=0 where username="{a}" and ac_no="{b}"'.format(a=name, b=id_nos))
                        cur.execute('update user_table set cur_balance=0 where username="{a}" and ac_no="{b}"'.format(a=name, b=id_nos))
                        conn.commit() # Make the changes permanent
                        print('\033[1mThanks for being a member in our bank\033[0m') 
                        print('Your account number is: {a}.\nPlease remember it for further communication\n'.format(a=id_nos))
                        print('For login please enter 2\n')
                        break
                    except:
                        # Condition to check number of wrong attempts and exist from the loop if exceeds the given limit 
                        if reg_pass_attempt<2: 
                            # Confirm if user wants to try again or exit from the process
                            reg_ch = input('Duplicate entry.\n If you want to try again please type "Y" else "N":')
                            if reg_ch=='Y':
                                reg_pass_attempt = reg_pass_attempt+1 # perform increment in the count of wrong attempts
                                pass
                            else:
                                break
                        else:
                            print('Too many attempts. Please try some other time.')
                            return None, None, None


                else:  
                    # Condition to check number of wrong attempts in providing the password and exist from the loop if exceeds the given limit 
                    if reg_pass_attempt<2:
                        # Confirm if user wants to try again or exit from the process
                        reg_ch = input('Your password was not of 4 digit in length.\n If you want to try again please type "Y" else "N":')
                        if reg_ch=='Y':
                            reg_pass_attempt = reg_pass_attempt+1
                            pass
                        else:
                            break
                    else:
                        print('Your password was not of 4 digit in length.\nToo many attempts.\nPlease restart the process again.')
                        return None, None, None           
        elif reg_ch1=='2':# loop for customer login
            while True:
                # Get the user name
                name = input('Enter your full name=\n')
                try:
                    # Check the database for the username
                    cur.execute('select count(*) from user_table where username="{a}"'.format(a=name))
                    if cur.fetchone()[0]>0: # if exist in the database than get the password
                        passwd = input('Enter your 4 digit password=\n')
                        if len(passwd)==4: # loop if the password is of correct lenght
                            # Check if the account exits in the database
                            cur.execute('select count(*) from user_table where username="{a}" and password="{b}"'.format(a=name, b=passwd))           
                            if cur.fetchone()[0]==0: # loop when account does't exist in the database
                                cur.execute('select login_fail from user_table where username="{a}"'.format(a=name))
                                # Check if the account is freezed due to too many wrong attempts
                                if cur.fetchone()[0] < 3:
                                    cur.execute('update user_table set login_fail=login_fail+1 where username="{a}"'.format(a=name))
                                    conn.commit()
                                    reg_ch = input('Your password did not matched.\n If you want to try again please type "Y" else "N":')
                                    if reg_ch=='Y':
                                        pass
                                    else:
                                        break  
                                else:
                                    print('Too many attempts. Please login again.')
                                    break
                            else: # When account exists in the database
                                # Check if the account name is freezed due to too many wrong attempts
                                cur.execute('select login_fail from user_table where username="{a}"'.format(a=name))
                                if cur.fetchone()[0] < 3:
                                    print('\033[1mYou have entered your bank login\033[0m')
                                    cur.execute('select ac_no from user_table where username="{a}" and password="{b}"'.format(a=name, b=passwd))
                                    # get the assigned user account number
                                    customer_id = cur.fetchone()[0]
                                    return name, passwd, customer_id
                                else:
                                    print('Account has been freezed. Please contact the nearest bank.')
                                    return None, None, None
                        # if the password length does't macthes the required length
                        else:
                            # Get the total failed login attempts
                            cur.execute('select login_fail from user_table where username="{a}"'.format(a=name))
                            if cur.fetchone()[0] < 3:
                                # Confirm if user wants to try again or exit from the process
                                reg_ch = input('Your password length did not matched.\n If you want to try again please type "Y" else "N":')
                                # update the failed login attempts in the database for the specific account
                                cur.execute('update user_table set login_fail=login_fail+1 where username="{a}"'.format(a=name))
                                conn.commit()
                                if reg_ch=='Y':
                                    pass
                                else:
                                    break      
                            else:
                                print('Too many attempts. Please contact the nearest bank and then login again.')
                                return None, None, None

                    # if the user name is not in the existing database
                    else:
                        if user_attempt < 2:
                            print('User name does not exists.')
                            # Confirm if user wants to try again or exit from the process
                            reg_ch = input('If you want to try again, please type "Y" else "N":')
                            if reg_ch=='Y':
                                user_attempt = user_attempt+1
                                pass
                            else:
                                break      
                        else:
                            print('Too many attempts. Please login again.')
                            return None, None, None 

                except:
                    print('Error during login in the database.')       
        # if use type an invalid key during identigying the choice input 
        else:
            print("Choice doesn't exist. \nPlease enter a valid choice")
            reg_ch = input('If you want to try again please type "Y" for exit press anything:')
            if reg_ch=='Y':
                pass
            else:
                break  
    return None, None, None

In [ ]:
def banking_operation(user_name,user_password,account_no,cur,conn):
    """
        Method Name: banking_operation
        Input:  user_name = User name,
                user_password = Account password,
                account_no = Account number,
                cur = Cursor,
                conn = Server connection
        Description: This function allows the regular user to do basic operations, whereas admin can use all functionality.
        Output: None
    """
    print('========================={a} good to see you============================================================\n'.format(a=user_name))
    print(dt.datetime.now())
    print('\n\033[1mPlease select the purpose of your login: \033[0m\n')
    while True:
        # Show all the opration the user can perform
        print('1.Modify Account Details\n')
        print('2.Perform Transaction (Only Admin)\n')
        print('3.Bank Statement\n')
        print('4.Delete Account (Only Admin)\n')
        print('5.Quit\n')
        # Get the user choice to perform specific task
        ch = input('Provide your input here:')
        if ch=='1':
            # create a account specific user table that stores all the personal details
            cur.execute("create table if not exists ac_{a}_user_table as select ac_no, username,address, phone_number, cur_balance from user_table where username='{b}' and password='{c}'".format(a=str(account_no), b=user_name, c=user_password))
            user_table_name = 'ac_{a}_user_table'.format(a=str(account_no))
            cur.execute('select * from {a}'.format(a=user_table_name))
            # Get all the column values from the table
            data = cur.fetchall()
            # Print all the personal details of the user
            for i in data:
                print('\033[1mAccount Number=\033[0m', i[0])
                print('\033[1mAccount Name=\033[0m', i[1])
                print('\033[1mCustomer Address =\033[0m', i[2])
                print('\033[1mCustomer Phone Number=\033[0m', i[3])
            while True:
                # Print all the available options in this tab
                print('\n\033[1mTo update the details please select the one of the following: \033[0m\n')
                print('1.Address\n')
                print('2.Phone Number\n')
                print('3.Quit\n')
                reg_ch = input('Your option:')
                if reg_ch=='1':
                    user_add = input('Please enter your address:')
                    # update the user address in all the tables
                    cur.execute("update {d} set address='{a}' where username='{b}' and ac_no={c}".format(a=user_add, b=user_name, c=account_no, d=user_table_name))
                    cur.execute("update user_table set address='{a}' where username='{b}' and ac_no={c}".format(a=user_add, b=user_name, c=account_no))
                    conn.commit()

                elif reg_ch=='2':
                    # Check if the phone number is in interger format else break from the ship the current loop
                    try:
                        user_ph = int(input('Please enter your phone number:'))
                    except:
                        print('\n\033[1mInvalid input\033[0m\n')
                        continue
                    # update the user address in all the tables     
                    cur.execute("update {d} set phone_number='{a}' where username='{b}' and ac_no={c}".format(a=str(user_ph), b=user_name, c=account_no, d=user_table_name))
                    cur.execute("update user_table set phone_number='{a}' where username='{b}' and ac_no={c}".format(a=str(user_ph), b=user_name, c=account_no))
                    conn.commit()

                elif reg_ch=='3': # lopp to exit back to main menu
                    print('Thanks for updating the details.\n')
                    break
                else: # loop if invalid key is provided in the choice
                    print('Invalid key!\n')
                    break

        elif ch=='2':
            # create account specfic tables if not existed previously
            cur.execute("create table if not exists ac_{a}_user_table as select ac_no, username,address, phone_number, cur_balance from user_table where username='{b}' and password='{c}'".format(a=str(account_no), b=user_name, c=user_password))
            cur.execute("create table if not exists ac_{a}_transaction_table as select ac_no, username, cur_balance from user_table where username='{b}' and password='{c}'".format(a=str(account_no), b=user_name, c=user_password))
            transaction_table_name = 'ac_{a}_transaction_table'.format(a=str(account_no)) 
            #cur.execute("alter table {a} drop primary key".format(a=transaction_table_name))
            #conn.commit()
            cur.execute("select * from {a}".format(a=transaction_table_name))
            # Get all the column names from the account specific transaction table
            fields = [field_md[0] for field_md in cur.description]
            date = ['debit_date', 'credit_date']
            amount = ['debit_amount', 'credit_amount']
            # add columns in the table, which were absent previously
            for d in date:
                if d not in fields:
                    cur.execute("alter table {a} add {b} datetime".format(a=transaction_table_name, b=d))
                    conn.commit()
            for m in amount:
                if m not in fields:
                    cur.execute("alter table {a} add {b} int".format(a=transaction_table_name, b=m))
                    conn.commit()
            cur.execute('select admin from user_table where username="{a}" and ac_no="{b}"'.format(a=user_name, b=account_no))
            # check is the user has admin rights
            if cur.fetchone()[0]=='True':
                while True:
                    # print avaliable options in this tab
                    print('\n\033[1mDo you wish to: \033[0m\n')
                    print('1.Deposit\n')
                    print('2.Withdrawal\n')
                    print('3.back\n')
                    reg_ch = input('Your option:')
                    if reg_ch=='1':
                        # check is provided value is in correct format
                        try:
                            credit = int(input('\nPlease enter the credit amount:'))
                        except:
                            print('\n\033[1mInvalid input\033[0m\n')
                            continue
                        cur.execute("select cur_balance from user_table")
                        # Update the existing balance
                        rev_balance = cur.fetchone()[0]+credit
                        # Update the balance in all the tables
                        cur.execute("update user_table set cur_balance=cur_balance+{a} where username='{b}' and ac_no={c}".format(a=credit, b=user_name, c=account_no))
                        cur.execute("update ac_{d}_user_table set cur_balance=cur_balance+{a} where username='{b}' and ac_no={c}".format(a=credit, b=user_name, c=account_no, d=account_no))
                        cur.execute("insert into {a} (ac_no, username, cur_balance, credit_date, credit_amount) values({b}, '{c}', {d}, '{e}', {f})".format(a=transaction_table_name, b=account_no, c=user_name, d=rev_balance, e=dt.datetime.today(), f=credit))
                        conn.commit()            
                    elif reg_ch=='2':
                        # check is provided value is in correct format
                        try:
                            debit = int(input('\nPlease enter the debit amount:'))
                        except:
                            print('\n\033[1mInvalid input\033[0m\n')
                            continue
                        cur.execute("select cur_balance from user_table")
                        # Check if enough balance present in the account for withdrawl
                        if cur.fetchone()[0]>int(debit):
                            cur.execute("select cur_balance from user_table")
                            # Update the existing balance
                            rev_balance = cur.fetchone()[0]-int(debit)
                            # Update the balance in all the tables
                            cur.execute("update user_table set cur_balance=cur_balance-{a} where username='{b}' and ac_no={c}".format(a=debit, b=user_name, c=account_no))
                            cur.execute("update ac_{d}_user_table set cur_balance=cur_balance-{a} where username='{b}' and ac_no={c}".format(a=debit, b=user_name, c=account_no, d=account_no))
                            cur.execute("insert into {a} (ac_no, username, cur_balance, debit_date, debit_amount) values({b}, '{c}', {d}, '{e}', {f})".format(a=transaction_table_name, b=account_no, c=user_name, d=rev_balance, e=dt.datetime.today(), f=debit))
                            conn.commit()  
                        else:
                            print('Not enouth balance in the account')
                    # loop to exit the current menu
                    elif reg_ch=='3':
                        break
                    # loop for invalid choice input
                    else:
                        print('\033[1mInvalid input!\033[0m\n')
                        break
            else:
                print("\033[1mYou don't have the right to perform access the transaction part. Please take admin help.\n\033[0m")
        # loop for getting the bank statement
        elif ch=='3':
            # create and check if transaction have been performed from this account
            try:
                cur.execute("create or replace view customer_view as select * from ac_{a}_transaction_table".format(a=account_no))
            except:
                print('No transaction avaliable for this account')
                continue
            cur.execute('select * from customer_view')
            # Get all the table values essential for the bank statement
            data = cur.fetchall()
            # Get all the column names of the table
            fields = [field_md[0] for field_md in cur.description]
            # Create a dataframe from the extracted data
            df = pd.DataFrame(data, columns=fields)
            # Save the dataframe in CSV format
            df.to_csv('E:/sql_project/bank_statement.csv', index=False)
            print(df)
        # loop to delete all the account specific table with admin approval
        elif ch=='4':
            cur.execute('select admin from user_table where username="{a}" and ac_no="{b}"'.format(a=user_name, b=account_no))
            # Check if admin status is avaliable for this user
            if cur.fetchone()[0]=='True':
                # Drop the user tables
                cur.execute("drop table if exists ac_{a}_user_table".format(a=account_no))
                cur.execute("drop table if exists ac_{a}_transaction_table".format(a=account_no))
                # Mark the user account with closed status
                cur.execute("update user_table set status='closed' where username='{a}' and ac_no={b}".format(a=user_name, b=account_no))
                conn.commit()
            else:
                print("\033[1mYou don't have the right to delete the database. Please take admin help.\n\033[0m")
        # exit from the main menu
        elif ch=='5':
            break
        else: # Loop for invalid key input
            print('\n\033[1mInvalid input\033[0m\n')
            break


In [ ]:
def make_admin(user_name, account_no):
    """
        Method Name: make_admin
        Input:  user_name = User name,
                account_no= Account number,
        Description: This function gives the user admin status.
        Output: None
    """
    cur.execute("update user_table set admin='True' where username='{a}' and ac_no='{b}'".format(a=user_name, b=account_no))
    conn.commit()

In [ ]:
# Make connection to the server
conn = sql.connect(host='localhost', user='root', passwd='Cct088014@')
cur = conn.cursor()
# Create database
cur.execute('create database if not exists bank')
# Use database
cur.execute('use bank')
# In the absence of user table create one
cur.execute('create table if not exists user_table(ac_no int not null, username varchar(50) not null, password varchar(25) not null, address varchar(30), phone_number varchar(15), cur_balance int, login_fail int, admin varchar(5), status varchar(20), primary key (username))')
# call the function account_login
name, pas, ac = account_login(cur, conn)
# Check if variable ac is None
if ac is not None:
    c = input('Do you want to make this account an admin. If yes then type Y:')
    if c=='Y':
        make_admin()
    # Call function banking_operation to perform management operations
    banking_operation(name, pas, ac, cur, conn)